## Data Frame to Newick 

## Libraries 

In [26]:
# Import Python Packages and Set DataFrame Display Options

import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
#import pandasql as ps
import datetime
import math as math
pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',1000)
pd.set_option('compute.use_bottleneck', True)
pd.set_option('compute.use_numexpr', True)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [30]:
sub1=pd.read_csv('/Sample.csv') ## read the file
sub_tble =sub1.pivot(index="child",columns='parent',values='Incident').fillna(0)

In [32]:
##################************ Table Mapping child and parent ***************##################
sub_tble =sub1.pivot(index="child",columns='parent',values='Incident').fillna(0)
leaf=sub_tble.index[~sub_tble.index.isin(sub_tble.columns)] ## finding thr bottom of the tree


##############*************DATA FRAME DF FOR ALL THE PATHS OF THE TREE AS ROWS*********############
DF=pd.DataFrame()
DF['parent0']=leaf
for c in range(1,15):                         ### for loop for each level of tree
    pt=[]                                     ### declaring empyt list
    for i in DF['parent'+str(c-1)]:           ### loop taking all the elements in lower level
        if i in sub_tble.index:              
            test=sub_tble.loc[i,:]
            pt.append(test[test == 1].index[0])
        else:
            pt.append(0.000)
    if sum(pt)==0.0000:
        print('loop breaks at parent' + str(c))
        break    
    DF['parent'+str(c)]=pt
DF=DF.iloc[:, ::-1]## IF YOU WANT TO REVERSE THE ORDER OF COLUMNS
DF_list=DF.values.tolist() ## getting each row into a list

##############***********REMOVING 0S AND GETTING PATHS AS LISTS***************#################

brch_hd=[] ### list of numbers with master branch nodes
sub_lst=[] ### list of lists with members in a brach more than 3 ####**** Again subsetting****#######
for lst in DF_list:
    lst_nonzero = [x for x in lst if x != 0]
    if len(lst_nonzero)>3: 
        sub_lst.append(lst_nonzero)
        #st_nonzero = [x for x in lst if x != 0][-1]  ## gets the last element of each row
        brch_hd.append(lst_nonzero[0])
brch_hd_series = pd.Series( (v for v in brch_hd))## getting the numbers for brnch head
brch_hd_series=brch_hd_series.unique() ## getting all the unique branch heads
brch_hd_unq_lst=brch_hd_series.tolist() ##putting them to a lsit

##############***** PUTTING THE LISTS AS ROWS TO A DATAFRAME*******##############
all_bunches=[]
for hd in brch_hd_unq_lst:
    bch_pth=[]
    for l in sub_lst:
        if hd in l:
            bch_pth.append(l)
    bch_df=pd.DataFrame(bch_pth)
    all_bunches.append(bch_df)
#bch_df=bch_df.iloc[:, ::-1]

loop breaks at parent11


In [34]:
def str_clean(x):
    x = x.replace('[', '(').replace('],', ')').replace(']', ')').replace("'", "").replace(" ", "").replace('"', '').replace("\\", "") 
    return x[1:-1]
string_bucket=[]
for bch_df in all_bunches:
    bch_df=bch_df.astype(str)
    n=bch_df.shape[1]-1
    for i in range(n):
        temp=pd.DataFrame()
        bch_df=bch_df.astype(str)
        #print('bch_df')
        #print(bch_df)
        temp=bch_df[~(bch_df[n-i]=='nan')]
        temp=temp.groupby(n-i-1,as_index=True)[n].unique().apply(list).to_frame().reset_index().iloc[:, ::-1]
        temp.columns=['c','p']
        temp['merge']=temp.iloc[:].values.tolist()
        temp=temp.astype(str)
        #print('temp')
        #print(temp)

        luck=pd.merge(bch_df,temp, how='left', left_on=[n-i-1],right_on=['p'],left_index=False, right_index=False, 
                 suffixes=('_x', '_y'), copy=True, indicator=False,validate=None)
        luck=luck.drop(columns=['c','p'])
        luck=luck.astype(str)
        #print('luck')
        #print(luck)

        bch_df[n]=luck['merge']
        bch_df=bch_df.astype(str)
        bch_df[n]=bch_df[n].apply(str_clean)
        for t in range(bch_df[n].size): 
            if bch_df[n][t]=='a':
                bch_df[n][t] = bch_df[n-i-1][t]
    string_bucket.append(bch_df[n][0])


final_tree=','.join(string_bucket)
final_tree='(('+final_tree+')Root);'

In [35]:
from ete3 import Tree

t = Tree(final_tree, format=1)
print( t.get_ascii(show_internal=True))


                         /-87.0
                        |
                   /1974.0-933.0
                  |     |
       /1839.01854.0     \-502415.0
      |           |
      |           |      /-1254.0
      |            \2275.0
      |                  \-4331.0
      |
      |                                               /-297.0
      |                                              |
      |                                         /2287.0-1013.0
      |                                        |     |
      |                                   /2239.0     \3238.0500732.0-503502.0
      |                                  |     |
      |                       /2044.02029.0     \3109.0-1406.0
      |                      |           |
      |                      |            \-500943.0
      |                 /1585.0
      |                |     |--1565.0
      |                |     |
      |                |      \1558.0-4765.0
      |                |
      |                |     